In [1]:
import sys
sys.path.append("../../src")
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.layers import Activation, Dropout, Lambda, Dense
from tensorflow.keras import Sequential
import os
import pandas as pd
from util import *
from model_trans import *
import json

In [2]:
with open("../../config/age_parameters.json") as param:
    parameters = json.load(param)["load_data"]
param.close()



In [3]:
data_generator, df = make_generator(**parameters)
train_idx, valid_idx, test_idx = data_generator.generate_split_indexes()

Number of training data:79135
Number of validation data:8793
Number of testing data:9770


In [ ]:
train = pd.read_csv("/teams/DSC180A_FA20_A00/a01capstonegroup03/fairface_label_train.csv")
train

In [ ]:
df["file"] = df["file"].apply(lambda x: "/".join(x.split("/")[-2:]))

In [ ]:
idx = df["file"].apply(lambda x:x.replace("/", "_").split(".")[0]).values

In [ ]:
df = df.set_index(idx)

In [4]:
df

,file,age,age_id
train_1,/teams/DSC180A_FA20_A00/a01capstonegroup03/fai...,50-59,6
train_2,/teams/DSC180A_FA20_A00/a01capstonegroup03/fai...,30-39,4
train_3,/teams/DSC180A_FA20_A00/a01capstonegroup03/fai...,3-9,1
train_4,/teams/DSC180A_FA20_A00/a01capstonegroup03/fai...,20-29,3
train_5,/teams/DSC180A_FA20_A00/a01capstonegroup03/fai...,20-29,3
...,...,...,...
val_10950,/teams/DSC180A_FA20_A00/a01capstonegroup03/fai...,30-39,4
val_10951,/teams/DSC180A_FA20_A00/a01capstonegroup03/fai...,50-59,6
val_10952,/teams/DSC180A_FA20_A00/a01capstonegroup03/fai...,60-69,7
val_10953,/teams/DSC180A_FA20_A00/a01capstonegroup03/fai...,20-29,3


In [ ]:
train[train["file"].str.contains("8895")]

In [ ]:
df.iloc[8895]

In [ ]:
test_idx[:128]

In [ ]:
sample = next(iter(test_generator))

In [ ]:
plt.imshow(sample[0][2])

In [5]:
model = keras.models.load_model("../../models/xception_tr/xception_tr.hdf5")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [6]:
test_batch_size = 128
test_generator = data_generator.generate_images(test_idx, is_training=False, batch_size=test_batch_size)
age_pred = model.predict(test_generator, steps=len(test_idx)//test_batch_size)

In [7]:
test_generator = data_generator.generate_images(test_idx, is_training=False, batch_size=test_batch_size)
samples = 0
images, age_true = [], []
for test_batch in test_generator:
    image = test_batch[0]
    labels = test_batch[1]
    images.extend(image)
    age_true.extend(labels[0])

age_true = np.array(age_true)


age_true, age_pred = age_true.argmax(axis=-1), age_pred.argmax(axis=-1)

In [8]:
age_pred

array([4, 4, 4, ..., 2, 5, 4])

In [9]:
age_true

array([4, 3, 6, ..., 1, 4, 4])

In [ ]:
(age_true == age_pred).mean()

In [ ]:
test_generator_1 = data_generator.generate_images(test_idx, is_training=False, batch_size=test_batch_size)

In [ ]:
sample = next(iter(test_generator))

In [ ]:
test_idx[:128]

In [ ]:
prediction = model.predict(sample)

In [ ]:
prediction.argmax(axis = 1)

In [ ]:
test_idx[:128]

In [ ]:
sample[1][0].argmax(axis = 1)

In [ ]:
plt.imshow(plt.imread("/teams/DSC180A_FA20_A00/a01capstonegroup03/fairface_pad025/train/8895.jpg"))

In [ ]:
train = pd.read_csv("/teams/DSC180A_FA20_A00/a01capstonegroup03/fairface_label_train.csv")

In [ ]:
train[train["file"].str.contains("35325")]

In [ ]:
plt.imshow(images[14])

In [ ]:
age_true

In [ ]:
age_pred

In [ ]:
(age_true == age_pred).mean()

In [ ]:
from sklearn.metrics import classification_report

cr_race = classification_report(age_true, age_pred, target_names=data_generator.dataset_dict['age_alias'].keys())
print(cr_race)